# Solutions of some confusions

orient='h' means boxplot is horizontal, palette="Set2" means colors

normally distributed= normal curve

80% relations are on weight and size.That's why weight and height are most important

threshold =3 means all the values between -3 to +3 will be kept and others will be removed

x= we are dropping weight from x variable.Other columns will be there except weight

y=we are selecting weight as a target variable




# HeadBrain use case 

1.Gender : 1 for male, 2 for female
    
2. Age range : 1 for ages 20-46, 2 for over 46
    
3. Head size: head size is cubic cm
    
4.Brain Weighr : brain weight is grams  
    
Our target variable is 'Brain Weight' which has continuous data . So, it's a regression problem (continuous=regression, catagorical=classification)       

# Importing required libraries

In [ ]:
#importing warning library to avoid any warning message
import warnings
warnings.simplefilter("ignore")

# Loading the dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
brain=pd.read_csv('headbrain.csv')
brain.head() #printing first 5 rows for dataset

In [ ]:
brain.shape #Checking data dimension

Q : Should I remove the outlier:
        
A : It needs to solve the problem using with outliers and without outliers. The best result will be accepted       

# Data preparation

In [ ]:
#Renaming the column names to bit easier terms
brain=brain.rename(columns={'AgeRange':'Age','HeadSize':'Size','BrainWeight':'Weight'})
brain.head()

In [ ]:
brain.describe() #ststistical summary

1. Here we can see there are no missing values

2. No much difference observed between mean and median so outliers are not there. Still we will validate further.

In [ ]:
brain.info()  #checking the datatype of all the columns

There is no catagorical column exists in this dataset. So, no encoding is needed.

In [ ]:
brain.isnull().sum()  #Checking if there is any null values or not

No null value in the dataset

# EDA

# Univariate analysis

Analysis the data for one column

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

sns.distplot(brain["Size"])
plt.title("Brain size")
plt.show()

It is showing that data is normally distributed in size column

In [ ]:
sns.distplot(brain["Weight"])
plt.title("Brain weight")
plt.show()

It is showing that data is normally distributed in size column

In [ ]:
#Plotting the "Gender" column
sns.countplot(brain["Gender"])    #There are only two values.Thats's why we have used countplot.
plt.title("Age Range")
plt.show()

In [ ]:
#Plotting the "Age" column
sns.countplot(brain["Age"])    #There are only two values.Thats's why we have used countplot.
plt.title("Age Range")
plt.show()

Data distribution of "Age Range" column is showing normal between both the catagories

# Bivariate Analysis (two columns are for analysis)

In [ ]:
#Plotting scatter plot to check relationship between "Size" and "Weight"
plt.scatter(brain["Size"],brain["Weight"])
plt.title("Scatter plot")
plt.xlabel("Size")
plt.ylabel("Weight")
plt.show()

There is positive linear relation above the scatter plot.

# Multivariate Analysis

In [ ]:
#Checking correlation using heatmap
corr=brain.corr()  #brain.corr() function provies the correlation value of each column
sns.heatmap(corr,annot=True) #Using heatmap we are plotting correlation values obtained by brain.corr() function
plt.show()

1. Very low correlation of "Age" with target variable so we can drop this column

2.Very high and positive correlation of "Size" with the target variable

3.Very negative correlation of "Gender" column with target variable but we will keep it as relation is strong

# Handling outliers

Outliers can impact the mean significantly. So it's always better to check the dataset for outliers and treat them if needed

In [ ]:
#Checking outliers using boxplot
sns.boxplot(data=brain[["Size","Weight"]],orient="h",palette="Set2")
plt.title("Outliers detection using Boxplot")
plt.show()

There are outliers which can be removed using different methods. In this project we will use Inter Quartile Range to remove the outliers

In [ ]:
from scipy.stats import zscore
import numpy as np
z=np.abs(zscore(brain))
threshold=3
new_brain=brain[(z<3).all(axis=1)]

In [ ]:
print("Original shape=",brain.shape,"\n","New shape : ", new_brain.shape)

# Skewness is also an important factor which impacts distribution of the data.Always try to make the dataset less skewed

In [ ]:
#Checking the skewness(if exists)
new_brain.skew()

Skewness is in acceptable range (+/- 0.5) so our data is not skewed

# Preparing dataset for model training

In [ ]:
#seperating independent and target variable
x=new_brain.drop("Weight",axis=1)
y=new_brain["Weight"] #Target variable

# Scaling

Model can be bised to higher values in dataset so it's betetr to scale the dataset so that we can bring all the column in common range. There are two algorithms available for scaling,StandardScaler and MinMaxScaler. We are going to use StandardScaler here  (MinMax is for range)

In [ ]:
#Scaling the dataset using SrandardScaler
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scaledx=sc.fit_transform(x)

In [ ]:
#findinfg the best random state
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

best_rstate=0
accu=0
for i in range(30,200):
    x_train,x_test,y_train,y_test=train_test_split(scaledx,y,test_size=.25,random_state=i)
    mod=LinearRegression()
    mod.fit(x_train,y_train)
    y_pred=mod.predict(x_test)
    tempaccu=r2_score(y_test,y_pred)   #What is the r2_score for particular random state
    if tempaccu>accu:
        accu=tempaccu
        best_rstate=i
        
print(f"Best accuracy{accu*100} found on random_state{best_rstate}")        

We found best random state 128 and will be using it in train_test_split in the next step

In [ ]:
# Creating train_test_split using best random_state
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(scaledx,y,test_size=.25,random_state=128)

# Finding the best model

LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
LR=LinearRegression()
LR.fit(x_train,y_train)
y_pred=LR.predict(x_test)
r2score=r2_score(y_test,y_pred)
cvscore=cross_val_score(LinearRegression(),x_train,y_train,cv=5).mean()
print(f"Accuracy={r2score*100},cross_val_score={cvscore*100} & difference={(r2score*100)-(cvscore*100)}")

RandomForestRegression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF=RandomForestRegressor()
RF.fit(x_train,y_train)
y_pred=RF.predict(x_test)
r2score=r2_score(y_test,y_pred)
cvscore=cross_val_score(RandomForestRegressor(),x_train,y_train,cv=5).mean()
print(f"Accuracy={r2score*100},cross_val_score={cvscore*100} & difference={(r2score*100)-(cvscore*100)}")

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ADB=AdaBoostRegressor()
ADB.fit(x_train,y_train)
y_pred=ADB.predict(x_test)
r2score=r2_score(y_test,y_pred)
cvscore=cross_val_score(AdaBoostRegressor(),x_train,y_train,cv=5).mean()
print(f"Accuracy={r2score*100},cross_val_score={cvscore*100} & difference={(r2score*100)-(cvscore*100)}")

SGDRegressor

In [ ]:
from sklearn.linear_model import SGDRegressor
SGD=SGDRegressor()
SGD.fit(x_train,y_train)
y_pred=SGD.predict(x_test)
r2score=r2_score(y_test,y_pred)
cvscore=cross_val_score(SGDRegressor(),x_train,y_train,cv=5).mean()
print(f"Accuracy={r2score*100},cross_val_score={cvscore*100} & difference={(r2score*100)-(cvscore*100)}")

LinearRegression and SGDRegressor are best performing model with almost same accuracy and cross valifdation score. We can choose either of them as our best model so I am moving ahead with SGDRegressor

# Hyperparameter tuning

It's technique to find out the best parameter for our model to improve the accuracy

In [ ]:
#Creating parameter list to pass the GridSearchCV
parameters = {"max_iter":[500,700,900,1100,1300,1400,1500],     #maximum iteration.It means how many times I will pass the data for iteration.
              
              "alpha":[0.0001,0.001,0.01,0.1,1,10,100],
              "penalty":["elasticnet","l1","l2"]}           #The best parameter will be selected automatically. We can use this code everywhere

In [ ]:
from sklearn.model_selection import GridSearchCV
GCV=GridSearchCV(SGDRegressor(),parameters,cv=5,scoring='r2')      #Initializing GridSearchCV cv=cross_validation ,r2 is for improving the accuracy
GCV.fit(x_train,y_train)                                           #cv=5 means 5 cross folds will be created and there will be training and testing data . Any fold will not be empty.
GCV.best_estimator_  #finding the best estimator
GCV_pred=GCV.best_estimator_.predict(x_test) #Predicting the value using best estimator found by GridSearchCV
print("Final accuracy : ",r2_score(y_test,GCV_pred)*100) #Final accuracy

Model accuracy is not increasing after 76.48%.It is because of less data. We need to collect more data to improve the accuracy further

# Serialization

In [ ]:
import joblib
joblib.dump(GCV.best_estimator_,"HeadBrainProject.pkl") #This trained model has been saved for production